# EDA: Amazon India (2015–2025)

This notebook performs exploratory data analysis on the cleaned CSVs under `data/cleaned/`.
It covers yearly revenue trends, seasonality, RFM segmentation, payment mix evolution,
category performance, Prime impact, geographic summaries, price vs demand, and delivery performance.

In [ ]:
import os, sys
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Plot styling
sns.set(style='whitegrid', palette='tab10')
plt.rcParams['figure.figsize'] = (8, 4)

def find_root(start=None):
    p = Path(start or os.getcwd()).resolve()
    for _ in range(8):
        if (p / 'pyproject.toml').exists() or ((p/'src').exists() and (p/'data').exists()):
            return str(p)
        p = p.parent
    return str(Path(start or os.getcwd()).resolve())

ROOT = find_root()
SRC = os.path.join(ROOT, 'src')
if SRC not in sys.path: sys.path.insert(0, SRC)
print('PROJECT_ROOT =', ROOT)


In [ ]:
# Load all cleaned CSVs
from pathlib import Path
cleaned_dir = os.path.join(ROOT, 'data', 'cleaned')
if not os.path.isdir(cleaned_dir):
    # Try fallback to ROOT/data or search upwards for data/cleaned
    alt = os.path.join(ROOT, 'data')
    if os.path.isdir(alt):
        cleaned_dir = alt
    else:
        p = Path(os.getcwd()).resolve()
        found = None
        for _ in range(8):
            cand = p / 'data' / 'cleaned'
            if cand.is_dir():
                found = str(cand)
                break
            p = p.parent
        if found:
            cleaned_dir = found
        else:
            raise FileNotFoundError(f'Could not find cleaned dir under {ROOT}; checked {alt} and ancestors.')
frames = []
for name in sorted(os.listdir(cleaned_dir)):
    if name.lower().endswith('.csv'):
        p = os.path.join(cleaned_dir, name)
        try:
            df_part = pd.read_csv(p)
            df_part['__source_file__'] = name
            frames.append(df_part)
        except Exception as e:
            print('Skip', name, '->', e)

df = pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
print('Using cleaned_dir =', cleaned_dir)
print('Rows:', len(df), 'Cols:', list(df.columns)[:10], '...')


In [ ]:
# Column detection helpers
DATE_CANDIDATES = ['order_date','date','orderDate','Order Date','order_dt']
REVENUE_CANDIDATES = ['total_amount','original_price_inr','revenue','amount','total','grand_total']
QUANTITY_CANDIDATES = ['quantity','qty','units']
CUSTOMER_CANDIDATES = ['customer_id','cust_id','user_id']
CATEGORY_CANDIDATES = ['category','Category']
PAYMENT_CANDIDATES = ['payment_method','payment','paymentType']
CITY_CANDIDATES = ['customer_city','city']
STATE_CANDIDATES = ['state','customer_state']
PRIME_CANDIDATES = ['is_prime_member','prime','isPrime']
RATING_CANDIDATES = ['customer_rating','rating','ratings']
DELIVERY_CANDIDATES = ['delivery_days','deliveryDays']
BRAND_CANDIDATES = ['brand','Brand']

def pick_col(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def coerce_date(series: pd.Series) -> pd.Series:
    s = series.astype('string')
    fmts = ['%Y-%m-%d','%d/%m/%Y','%d-%m-%y','%d-%m-%Y','%m/%d/%Y']
    parsed = pd.Series(pd.NaT, index=s.index, dtype='datetime64[ns]')
    mask = parsed.isna()
    for fmt in fmts:
        if not mask.any():
            break
        rem = s[mask]
        p = pd.to_datetime(rem, errors='coerce', format=fmt)
        good = p.notna()
        if good.any():
            parsed.loc[good.index[good]] = p[good]
            mask = parsed.isna()
    if mask.any():
        rem = s[mask]
        p = pd.to_datetime(rem, errors='coerce')
        good = p.notna()
        if good.any():
            parsed.loc[good.index[good]] = p[good]
    return parsed

date_col = pick_col(df, DATE_CANDIDATES)
revenue_col = pick_col(df, REVENUE_CANDIDATES)
customer_col = pick_col(df, CUSTOMER_CANDIDATES)
category_col = pick_col(df, CATEGORY_CANDIDATES)
payment_col = pick_col(df, PAYMENT_CANDIDATES)
city_col = pick_col(df, CITY_CANDIDATES)
state_col = pick_col(df, STATE_CANDIDATES)
prime_col = pick_col(df, PRIME_CANDIDATES)
rating_col = pick_col(df, RATING_CANDIDATES)
quantity_col = pick_col(df, QUANTITY_CANDIDATES)
brand_col = pick_col(df, BRAND_CANDIDATES)

print('Detected columns:')
for k,v in [('date',date_col),('revenue',revenue_col),('customer',customer_col),('category',category_col),('payment',payment_col),('city',city_col),('state',state_col),('prime',prime_col),('rating',rating_col),('quantity',quantity_col),('brand',brand_col)]:
    print(f' - {k}:', v)

# Ensure revenue numeric
if revenue_col: df[revenue_col] = pd.to_numeric(df[revenue_col], errors='coerce')


## 1) Yearly Revenue Trend

In [ ]:
if date_col and revenue_col and not df.empty:
    dt = coerce_date(df[date_col])
    d = df.copy(); d['year'] = dt.dt.year
    rev = d.groupby('year')[revenue_col].sum().reset_index()
    rev = rev[(rev['year']>=2015)&(rev['year']<=2025)]
    rev['pct_growth'] = rev[revenue_col].pct_change()*100
    if len(rev) >= 2:
        z = np.polyfit(rev['year'], rev[revenue_col], 1)
        p = np.poly1d(z)
        rev['trend'] = p(rev['year'])
    ax = sns.lineplot(data=rev, x='year', y=revenue_col, marker='o', label='Revenue')
    if 'trend' in rev.columns:
        sns.lineplot(data=rev, x='year', y='trend', linestyle='--', label='Trend', ax=ax)
    plt.title('Yearly Revenue (2015–2025)')
    plt.show()
    rev.head()


## 2) Seasonality: Monthly Heatmap and Category Comparison

In [ ]:
if date_col and revenue_col and not df.empty:
    dt = coerce_date(df[date_col])
    t = df.copy(); t['year'] = dt.dt.year; t['month'] = dt.dt.month
    pivot = t.pivot_table(index='year', columns='month', values=revenue_col, aggfunc='sum', fill_value=0)
    sns.heatmap(pivot, cmap='YlGnBu'); plt.title('Monthly Sales Heatmap'); plt.show()
    pivot.head()
    if category_col and category_col in df.columns:
        cat = t.pivot_table(index='month', columns=category_col, values=revenue_col, aggfunc='sum', fill_value=0)
        cat.plot(); plt.title('Monthly Seasonality by Category'); plt.xlabel('Month'); plt.ylabel('Revenue'); plt.show()
        cat.head()


## 3) RFM Segmentation

In [ ]:
if date_col and customer_col and revenue_col and not df.empty:
    d = df[[date_col, revenue_col, customer_col]].dropna().copy()
    d[date_col] = coerce_date(d[date_col])
    ref = d[date_col].max()
    grp = d.groupby(customer_col)
    R = grp[date_col].max().apply(lambda x: (ref - pd.to_datetime(x)).days if pd.notna(x) else np.nan)
    F = grp[date_col].count()
    M = grp[revenue_col].sum()
    rfm = pd.DataFrame({'R':R,'F':F,'M':M}).replace([np.inf,-np.inf],np.nan).dropna()
    def qscores(s, buckets=5, reverse=False):
        ranks = s.rank(pct=True, method='first')
        if reverse: ranks = 1 - ranks
        return np.ceil(ranks*buckets).astype(int).clip(1,buckets)
    rfm['R_score'] = qscores(rfm['R'], reverse=True)
    rfm['F_score'] = qscores(rfm['F'])
    rfm['M_score'] = qscores(rfm['M'])
    plt.scatter(rfm['F'], rfm['M'], c=rfm['R'], cmap='viridis', alpha=0.6)
    plt.colorbar(label='Recency (days)'); plt.xlabel('Frequency'); plt.ylabel('Monetary'); plt.title('RFM Scatter'); plt.show()
    rfm.head()


## 4) Payment Method Evolution

In [ ]:
if date_col and payment_col and revenue_col and not df.empty:
    dt = coerce_date(df[date_col])
    t = df.copy(); t['year'] = dt.dt.year
    mix = t.groupby(['year', payment_col])[revenue_col].sum().reset_index()
    total = mix.groupby('year')[revenue_col].transform('sum')
    mix['share'] = mix[revenue_col]/total
    piv = mix.pivot(index='year', columns=payment_col, values='share').fillna(0)
    piv.plot(kind='area', stacked=True, colormap='tab20'); plt.title('Payment Method Share Over Time'); plt.ylabel('Share'); plt.show()
    mix.head()


## 5) Category Performance

In [ ]:
if date_col and category_col and revenue_col and not df.empty:
    dt = coerce_date(df[date_col])
    t = df.copy(); t['year'] = dt.dt.year
    agg = t.groupby(['year', category_col])[revenue_col].sum().reset_index()
    latest = int(agg['year'].max()) if not agg.empty else None
    if latest is not None:
        cur = agg[agg['year']==latest].sort_values(revenue_col, ascending=False)
        sns.barplot(data=cur, x=revenue_col, y=category_col); plt.title(f'Category Revenue - {latest}'); plt.show()
        agg.head()


## 6) Prime Impact

In [ ]:
if date_col and prime_col and revenue_col and not df.empty:
    t = df.copy(); t['prime'] = t[prime_col].astype('boolean')
    aov = t.groupby('prime')[revenue_col].mean().reset_index()
    sns.barplot(data=aov, x='prime', y=revenue_col); plt.title('Average Order Value by Prime'); plt.show()
    if customer_col and customer_col in t.columns:
        orders = t.groupby([customer_col, 'prime']).size().reset_index(name='orders')
        freq = orders.groupby('prime')['orders'].mean().reset_index()
        sns.barplot(data=freq, x='prime', y='orders'); plt.title('Avg Orders per Customer by Prime'); plt.show()
    aov.head()


## 7) Geographic (Top Cities)

In [ ]:
if city_col and revenue_col and not df.empty:
    top = df.groupby(city_col)[revenue_col].sum().reset_index().sort_values(revenue_col, ascending=False).head(20)
    sns.barplot(data=top, x=revenue_col, y=city_col); plt.title('Top Cities by Revenue'); plt.show()
    top.head()


## 10) Price vs Demand and Correlation

In [ ]:
if revenue_col and quantity_col and not df.empty:
    price_col = 'unit_price' if 'unit_price' in df.columns else revenue_col
    sns.scatterplot(data=df, x=price_col, y=quantity_col, hue=category_col if category_col in df.columns else None, alpha=0.3)
    plt.title('Price vs Demand'); plt.show()
    cols = [c for c in [price_col, quantity_col] if c in df.columns]
    if len(cols)>=2:
        corr = df[cols].corr()
        sns.heatmap(corr, annot=True, vmin=-1, vmax=1, cmap='coolwarm'); plt.title('Correlation'); plt.show()
        corr


## 11) Delivery Performance

In [ ]:
if 'delivery_days' in df.columns:
    sns.histplot(df['delivery_days'].dropna(), bins=30); plt.title('Delivery Days Distribution'); plt.show()
    if rating_col and rating_col in df.columns:
        sns.scatterplot(data=df, x='delivery_days', y=rating_col, alpha=0.3); plt.title('Delivery vs Rating'); plt.show()
